In [4]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn.linear_model  as sk  # namespace conflict with pyspark
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import scale
from sklearn.model_selection import KFold , cross_val_score
from sklearn.preprocessing import StandardScaler
import pandas as pd
from helper import *
import tensorflow as tf

pd.options.display.max_columns = None
import findspark
import pyspark
from pyspark.sql import SQLContext
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.mllib.regression import LabeledPoint, LinearRegressionWithSGD
from pyspark.sql.functions import col

from pyspark.ml.linalg import VectorUDT
from pyspark.sql.functions import udf
from pyspark.ml.evaluation import RegressionEvaluator

from keras.layers import Dense
from keras.models import Model, Sequential
from keras import initializers, optimizers
from keras import backend as K

# Stuff that needs to be done

- Find another framework.
    - Sklearn
    - Spark
    - Tensorflow/R
- Algorithms
    - Linear Regression
    - Logistic Regression
    - Maybe SVM /Random forest
- Find two metrics 
    - RMSE 
    - Some other
- Find another dataset 
    - Sum data with noise 
    -  Maybe housing data set
    

## Import DataSet

In [5]:
sumdata_url = "https://www.dropbox.com/sh/euppz607r6gsen2/AABABUTdx7YqCeBquA1Ky7z8a/The%20SUM%20dataset?dl=1#"
housing_price_url = "https://www.dropbox.com/sh/euppz607r6gsen2/AAAVLZzU4E7ro0BiRzPG3pP8a/House%20Prices?dl=1"
all_urls = [sumdata_url, housing_price_url]

In [6]:
get_data(all_urls) # retrieves the data if there is no data folder

In [7]:
sumdata_noise_path = "data/with noise/The SUM dataset, with noise.csv"
sumdata_path = "data/without noise/The SUM dataset, without noise.csv"
housing_price_path = "data/housing dataset.csv" # has more than 30 features
 

## Sum Data set size 100,000

In [34]:
SUM_DATA_SIZE = 100000

## Load datasets sum_noise

In [35]:
sumdata_noise = pd.read_csv(sumdata_noise_path, delimiter=";")
sumdata_noise = sumdata_noise.head(n=SUM_DATA_SIZE)
sumdata_noise.head(n=1)

,Instance,Feature 1,Feature 2,Feature 3,Feature 4,Feature 5 (meaningless but please still use it),Feature 6,Feature 7,Feature 8,Feature 9,Feature 10,Noisy Target,Noisy Target Class
0,1,62485,58472,84200,86181,75529,136939,150633,230058,246491,257336,1352179,Very Large Number


## Preprocess sum_noise dataset

- Remove 'Instance' as it simply represents the row number
- Extract 'Nosiy Target' as regression target
- Extract 'Nosiy Class' as classification target
- Extract rest columns as explananatory variables
- Apply Feature Scaling to the dataset 

- Ensure all dataframe has been converted to numpy array


### Create same base line for Spark, Keras, Sklearn implementation
- Using the same feature scaled data for all implementation
- Same number iterations
- Same cost function for minimising(for gradient descent)

# Need to check the ordering of the categorical to numerical

In [36]:
# Remove 'Instance' as it simply represents the row number

from sklearn import preprocessing
le = preprocessing.LabelEncoder()

sumdata_noise.drop('Instance', axis = 1)

# Extract 'Nosiy Target' as regression target
sumdata_noise_reg_Y = sumdata_noise.loc[:, ['Noisy Target']]
# Extract 'Nosiy Target Class' as regression target
sumdata_noise_classif_Y = sumdata_noise.loc[:, ['Noisy Target Class']] 
 
 
 
sumdata_noise_classif_Y ['Noisy Target Class'] = le.fit_transform( sumdata_noise_classif_Y ['Noisy Target Class'].astype('str')) 
 
# Extract rest columns as explananatory variables
sumdata_noise_X =sumdata_noise.drop(['Instance','Noisy Target Class','Noisy Target'
                                    ], axis = 1) 
 
sumdata_noise_classif_Y.rename(columns={'Noisy Target Class': 'label'}, inplace=True)



### Feature scaling
- Keeping them as dataframes (transform returns ndarray)

In [37]:
scX = StandardScaler()
scY = StandardScaler()
columns = sumdata_noise_X.columns

sumdata_noise_X[columns] = scX.fit_transform(sumdata_noise_X)
columns = sumdata_noise_reg_Y.columns
sumdata_noise_reg_Y[columns] = scX.fit_transform(sumdata_noise_reg_Y)


sumdata_noise_scaled = pd.concat([sumdata_noise_X,sumdata_noise_reg_Y])

In [38]:
sc = pyspark.SparkContext()
sqlContext = SQLContext(sc)


ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by __init__ at <ipython-input-12-8030b2fc72d9>:1 

In [46]:
def spark_logistic_regression( X_train, X_test, y_train, y_test):
    """
        this uses the dataframe api
    """
    from pyspark.ml import Pipeline
    from pyspark.ml.classification import LogisticRegression
    from pyspark.ml.evaluation import MulticlassClassificationEvaluator
    # spark needs a target column
     
    
    
    ## SHOULD FIND A BETTER WAY
    X_train.reset_index(drop=True, inplace=True)
    y_train.reset_index(drop=True, inplace=True)
    
    X_test.reset_index(drop=True, inplace=True)
    y_test.reset_index(drop=True, inplace=True)
    
    train_data = pd.concat([X_train,y_train], axis =1)
    test_data  = pd.concat([X_test,y_test], axis =1)
    
 
    train_data = sqlContext.createDataFrame(train_data)
    test_data = sqlContext.createDataFrame(test_data) 
   
   
    assembler = (VectorAssembler()
        .setInputCols(train_data.columns[:-1]) # everything excluding target
        .setOutputCol("features"))
 
    lr = LogisticRegression(maxIter=10, regParam=0.01)

    pipeline = Pipeline(stages=[assembler, lr])
    model = pipeline.fit(train_data)
    
    
    predictions = model.transform(test_data) 
    evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction")
    accuracy = evaluator.evaluate(predictions, {evaluator.metricName: "accuracy"})
    precision = evaluator.evaluate(predictions, {evaluator.metricName: "weightedPrecision"})
    return (accuracy, precision )


In [47]:
def model_cross_valdiation( X, y, dataset_name, algorithm, kfolds,regression=False): 
    
    kf = KFold(n_splits=kfolds)
    rmse = np.zeros((kfolds,1))
    mae = np.zeros((kfolds,1))
    accuracy = np.zeros((kfolds,1))
    precision = np.zeros((kfolds,1))
    
    algorithm_n = algorithm.__name__ 
 

    for (i, (train_index, test_index)) in enumerate(kf.split(X)):


        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        

        # accumulating errors for each fold 
        # calculates mean across rows
        if regression:     
            rmse[i], mae[i] = algorithm( X_train, X_test, y_train, y_test)   # RMSE https://www.kaggle.com/wiki/RootMeanSquaredError
           
        else: 
            accuracy[i],precision[i] = algorithm( X_train, X_test, y_train, y_test)
           
    if regression:     
        print ("Dataset: {}\nRMSE: {}\nMAE: {}\n".format( dataset_name, rmse.mean(), mae.mean()))
    else:
        print ("Dataset: {}\Accuracy: {}\nPrecision: {}\n".format(dataset_name, accuracy.mean(), precision.mean()))
    
          

In [48]:
model_cross_valdiation( sumdata_noise_X, sumdata_noise_classif_Y, "Sum data with noise", spark_logistic_regression,10, True)


Dataset: Sum data with noise
RMSE: 0.95061
MAE: 0.932101779454789



### 70/30 split 

In [49]:
def model_test_split( X, y, dataset_name, algorithm, regression=False): 
    algorithm_n = algorithm.__name__
    rmse = 0
    mae = 0 
    accuracy = 0
    precision = 0  

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

    if regression:     
        rmse, mae = algorithm( X_train, X_test, y_train, y_test)   # RMSE https://www.kaggle.com/wiki/RootMeanSquaredError
        print ("Algorithm: {} Dataset: {}\nRMSE: {}\nMAE: {}\n".format( 
                                                                algorithm_n,
                                                                dataset_name,
                                                                rmse,
                                                                mae))
        
    else: 
        accuracy ,precision  = algorithm( X_train, X_test, y_train, y_test)
        # accumulating errors for each fold
        print ("Algorithm: {} Dataset: {}\Accuracy: {}\nPrecision: {}\n".format(
                                                                algorithm_n,
                                                                dataset_name,
                                                                accuracy,
                                                                precision))



In [50]:
model_test_split( sumdata_noise_X, sumdata_noise_classif_Y, "Sum data with noise", spark_logistic_regression)

Algorithm: spark_logistic_regression Dataset: Sum data with noise\Accuracy: 0.9498333333333333
Precision: 0.9312965956191901

